In [32]:
# in case export to pdf doens't work
#!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic
!pip install -U sentence-transformers
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 5.7 MB/s  0:00:04 eta 0:00:01


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import ast


In [9]:
def load_csv(file_path):
    return kagglehub.load_dataset(
        KaggleDatasetAdapter.PANDAS,
        "rounakbanik/the-movies-dataset",
        file_path,
    )

movies = load_csv("movies_metadata.csv")
credits = load_csv("credits.csv")
keywords = load_csv("keywords.csv")
links = load_csv("links.csv")
links_small = load_csv("links_small.csv")


/tmp/ipykernel_69981/1641172065.py:2: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  return kagglehub.load_dataset(
/home/igorg/.pyenv/versions/3.10.6/envs/floportop/lib/python3.10/site-packages/kagglehub/pandas_datasets.py:92: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(
/tmp/ipykernel_69981/1641172065.py:2: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  return kagglehub.load_dataset(
/tmp/ipykernel_69981/1641172065.py:2: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  return kagglehub.load_dataset(
/tmp/ipykernel_69981/1641172065.py:2: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  return kagglehub.load_dataset(
/tmp/

In [10]:
movies = movies[movies["id"].str.isnumeric()]
movies["id"] = movies["id"].astype(int)

credits["id"] = credits["id"].astype(int)
keywords["id"] = keywords["id"].astype(int)


In [11]:
links["imdbId"] = links["imdbId"].apply(lambda x: f"tt{x:07d}")
links_small["imdbId"] = links_small["imdbId"].apply(lambda x: f"tt{x:07d}")


In [12]:
df = (
    movies
    .merge(credits, on="id", how="left")
    .merge(keywords, on="id", how="left")
    .merge(links, left_on="id", right_on="tmdbId", how="left")
)
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,cast,crew,keywords,movieId,imdbId,tmdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",1,tt0114709,862.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",2,tt0113497,8844.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",3,tt0113228,15602.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",4,tt0114885,31357.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",5,tt0113041,11862.0


In [13]:
def parse_json(col):
    return col.apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])


In [14]:
df["genres"] = parse_json(df["genres"])
df["keywords"] = parse_json(df["keywords"])
df["cast"] = parse_json(df["cast"])
df["crew"] = parse_json(df["crew"])


In [15]:
df.sample()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,cast,crew,keywords,movieId,imdbId,tmdbId
33234,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,49693,tt0095511,fr,La lectrice,A multi-faceted film based on Raymond Jean's n...,...,The Reader,False,4.4,7.0,"[{'cast_id': 2, 'character': 'Constance / Mari...","[{'credit_id': '52fe47a1c3a36847f8140125', 'de...",[],141904,tt0095511,49693.0


In [16]:
df["genre_names"] = df["genres"].apply(
    lambda xs: [x["name"] for x in xs]
)


In [17]:
df['genre_names'].sample(4)

22669                                              [Drama]
45105                                                   []
38045                             [Drama, Science Fiction]
19363    [Animation, Adventure, Comedy, Family, Action,...
Name: genre_names, dtype: object

In [18]:
df["keyword_names"] = df["keywords"].apply(
    lambda xs: [x["name"] for x in xs]
)


In [19]:
df['keyword_names'].sample(4)

26681      [puppy, wedding anniversary, henpecked husband]
25413       [romance, dating, relationship, online dating]
22837    [alcohol, adultery, river, camping, province, ...
27771    [secret, insurance fraud, murder, gunfight, in...
Name: keyword_names, dtype: object

In [20]:
df["cast_top"] = df["cast"].apply(
    lambda xs: [x["name"] for x in xs[:10]]
)


In [21]:
df['cast_top'].sample(4)

12320    [Humphrey Bogart, Fredric March, Arthur Kenned...
35370                                                   []
2647     [Frank Silvera, Jamie Smith, Irene Kane, Jerry...
6315     [Paul Walker, Tyrese Gibson, Eva Mendes, Cole ...
Name: cast_top, dtype: object

In [22]:
df["directors"] = df["crew"].apply(
    lambda xs: [x["name"] for x in xs if x["job"] == "Director"]
)


In [23]:
df["year"] = pd.to_datetime(
    df["release_date"], errors="coerce"
).dt.year

df["runtime"] = pd.to_numeric(df["runtime"], errors="coerce")
df["vote_average"] = pd.to_numeric(df["vote_average"], errors="coerce")
df["vote_count"] = pd.to_numeric(df["vote_count"], errors="coerce")
df["popularity"] = pd.to_numeric(df["popularity"], errors="coerce")


In [24]:
wide_df = df[[
    "id",                 # TMDB ID
    "imdbId",             # IMDb ID (primary external key)
    "title",
    "original_title",
    "overview",
    "genre_names",
    "keyword_names",
    "cast_top",
    "directors",
    "original_language",
    "year",
    "runtime",
    "vote_average",
    "vote_count",
    "popularity",
]].reset_index(drop=True)


In [25]:
wide_df.sample(3)

,id,imdbId,title,original_title,overview,genre_names,keyword_names,cast_top,directors,original_language,year,runtime,vote_average,vote_count,popularity
2430,25748,tt0106557,Children of the Corn II: The Final Sacrifice,Children of the Corn II: The Final Sacrifice,"Arriving in Gatlin, Nebraska, a news-reporter ...",[Horror],"[sequel, possession, priest, corn, harvest]","[Terence Knox, Paul Scherrer, Ryan Bollman, Ch...",[David Price],en,1993.0,92.0,4.5,50.0,8.357875
14263,49982,tt0096908,A City of Sadness,悲情城市,Soon after Japan relinquishes control of Taiwa...,[Drama],[],"[Tony Leung Chiu-Wai, Sung Young Chen, Wu Nien...",[Hou Hsiao-hsien],zh,1989.0,157.0,7.5,25.0,1.216930
4020,28368,tt0093073,Gardens of Stone,Gardens of Stone,A sergeant must deal with his desires to save ...,"[Drama, History]","[vietnam veteran, washington d.c., cemetery, v...","[James Caan, Anjelica Huston, James Earl Jones...",[Francis Ford Coppola],en,1987.0,111.0,5.5,25.0,2.787833


In [26]:
wide_df["embedding_text"] = (
    "Title: " + wide_df["title"].fillna("") + "\n"
    "Overview: " + wide_df["overview"].fillna("") + "\n"
    "Genres: " + wide_df["genre_names"].apply(lambda x: ", ".join(x)) + "\n"
    "Keywords: " + wide_df["keyword_names"].apply(lambda x: ", ".join(x)) + "\n"
    "Cast: " + wide_df["cast_top"].apply(lambda x: ", ".join(x)) + "\n"
    "Director: " + wide_df["directors"].apply(lambda x: ", ".join(x))
)


In [27]:
wide_df["embedding_text"].sample(4)

18932    Title: One of Our Dinosaurs Is Missing\nOvervi...
1592     Title: Alive and Kicking\nOverview: A passiona...
6239     Title: Flickering Lights\nOverview: Four small...
22696    Title: Open Up to Me\nOverview: At first glanc...
Name: embedding_text, dtype: object

In [28]:
from sentence_transformers import SentenceTransformer
import numpy as np


In [29]:
model = SentenceTransformer("all-MiniLM-L6-v2")  # 384-dim, fast

texts = wide_df["embedding_text"].fillna("").tolist()

embeddings = model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True  # IMPORTANT for cosine similarity
)

embeddings = np.asarray(embeddings, dtype="float32")


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 206.70it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 733/733 [05:46<00:00,  2.11it/s]


In [33]:
import faiss


In [34]:
dim = embeddings.shape[1]

index = faiss.IndexFlatIP(dim)  # exact search, cosine sim
index.add(embeddings)

print("Indexed vectors:", index.ntotal)


Indexed vectors: 46911


In [36]:
def search_by_text(query, k=10):
    q_emb = model.encode(
        [query],
        normalize_embeddings=True
    ).astype("float32")

    scores, idxs = index.search(q_emb, k)

    results = wide_df.iloc[idxs[0]].copy()
    results["score"] = scores[0]

    return results[[
        "title",
        "year",
        "genre_names",
        "score"
    ]]


In [37]:

search_by_text(
    "dream within a dream sci-fi mind bending reality",
    k=10
)


,title,year,genre_names,score
14130,Blue Corner,2002.0,[],0.554816
3683,Dreamscape,1984.0,"[Adventure, Horror, Science Fiction, Thriller]",0.550329
39188,FLicKeR,2009.0,[Documentary],0.499165
34400,The Edge of Dreaming,2010.0,[],0.485867
46119,Let Me Dream Again,1900.0,"[Romance, Comedy]",0.482698
222,Dream Man,1995.0,[Thriller],0.477875
22889,Into the Mind,2013.0,[Documentary],0.477473
15598,Dream,2008.0,"[Drama, Science Fiction]",0.477407
46055,On Body and Soul,2017.0,"[Drama, Romance]",0.477340
46697,The Astronomer's Dream,1898.0,[Fantasy],0.477232
